In [42]:
import torch,os
import tensorflow as tf
import torchvision.transforms as transforms
import torchvision.datasets as dset

In [43]:
root = './data'
if not os.path.exists(root):
    os.mkdir(root)
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
# if not exist, download mnist dataset
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)

batch_size = 1000

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)
use_cuda = torch.cuda.is_available()
softmax=torch.nn.Softmax()

In [44]:
class MLP:
    def __init__(self,in_features,out_features):
        self.L=torch.nn.Sequential(
            torch.nn.Linear(in_features=in_features,out_features=256,bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(256,256,bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(256,out_features=out_features,bias=True)
            )
    def forward_pass(self,X):
        return softmax(self.L(X))
    
    def accuracy(self,test_dataset):
        c=0
        for i in range(test_dataset.batch_size):
            if(torch.argmax(self.forward_pass(torch.flatten(test_dataset.dataset[i][0])))==test_dataset.dataset[i][1]):
                c+=1
        return c/test_dataset.batch_size
    
def onehotencoding(y):
    z=torch.zeros(10)
    z[y]=1
    return z

def conv(y):
    t=torch.argmax(y)
    z=torch.zeros(10)
    z[t]=1
    return z
    

In [45]:
model=MLP(784,10)
learning_rate=0.01
loss=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD([{"params":[model.L[0].weight,model.L[2].weight,model.L[4].weight,model.L[0].bias,model.L[2].bias,model.L[4].bias]}],lr=learning_rate)
n_iters=1000
for epoch in range(n_iters):
    for pts in range(train_loader.batch_size):
        optimizer.zero_grad()
        y_pred=model.forward_pass(torch.flatten(train_loader.dataset[pts][0]))
        l=loss(y_pred,onehotencoding(train_loader.dataset[pts][1]))
        l.backward()
        optimizer.step()



C:\Users\prana\AppData\Local\Temp\ipykernel_21124\3033164988.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return softmax(self.L(X))


In [46]:
model.accuracy(test_loader)


C:\Users\prana\AppData\Local\Temp\ipykernel_21124\3033164988.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return softmax(self.L(X))


0.843